In [1]:
from rich import print
from datasets import load_dataset , load_metric
from transformers import AutoTokenizer , BatchEncoding, BartForConditionalGeneration, BartTokenizer
import nltk
import string
import torch
import evaluate
from tqdm import tqdm
import numpy as np
import pandas as pd
import json

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("json" , data_files="./data/test.json")
dataset

DatasetDict({
    train: Dataset({
        features: ['body'],
        num_rows: 13762
    })
})

In [4]:
%pwd

'/root/code/python/NYCU-Data-Science-2024/HW3'

In [5]:
# model_id = "facebook/bart-base"
model_id = "model/model/data_science_hw3_model_facebook-bart-large/checkpoint-6300"

In [6]:
tokenizer = BartTokenizer.from_pretrained(model_id)
tokenizer

BartTokenizer(name_or_path='model/model/data_science_hw3_model_facebook-bart-large/checkpoint-6300', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [7]:
model = BartForConditionalGeneration.from_pretrained(model_id)
model = model.to(device)
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [8]:
# 检查模型参数的设备
device = next(model.parameters()).device

# 打印设备信息
print(f"Model is running on {device}.")

Model is running on cuda:0.

In [9]:
file_name = "109511276.json"
with open(file=file_name , mode="w") as f:
    pass 

In [10]:
max_input_length = 1024
max_target_length = 64

# 将模型移到相同的设备上
# model.to(device)

for test_item in tqdm(dataset["train"], desc="check"):
    # 将输入张量移动到设备
    # label = tokenizer(test_item["headline"], max_length=max_target_length, truncation=True, padding=True, return_tensors="pt").to(device) 
    pre_text_input = tokenizer(test_item["body"], max_length=max_input_length, truncation=True, padding=True, return_tensors="pt").to(device) 
    
    # 生成预测
    with torch.no_grad():
        outputs = model.generate(
            **pre_text_input,
            num_beams=10,
            early_stopping=True,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            no_repeat_ngram_size=2,
        )
        
    out_to_json = tokenizer.decode(outputs[0].cpu() , skip_special_tokens=True)
    
    out_to_json_dict = {"headline" : out_to_json}
    with open(file=file_name , mode="a") as f:
        json.dump(out_to_json_dict , f, ensure_ascii=False)
        f.write("\n")


check:   0%|          | 0/13762 [00:00<?, ?it/s]

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
check: 100%|██████████| 13762/13762 [2:06:48<00:00,  1.81it/s] 


In [11]:
print(f"save file to : {file_name}")

save file to : 109511276.json